In [5]:
import torch
import torch.nn as nn
from transformers import AutoModel
import pandas as pd

import gc
import torch as th
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import transformers
import transformers as tfms
import torch.optim as optim
import matplotlib.pyplot as plt
from typing import Tuple
from datasets import load_dataset

torch.manual_seed(0)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
try:
  FLOAT = torch.bfloat32 if torch.cuda.is_bf16_supported() else \
    torch.float32
except Exception as e:
  print(e)
  FLOAT = torch.float32
print(DEVICE, FLOAT)

CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.
cpu torch.float32


In [6]:
# config = tfms.DistilBertConfig.from_pretrained('hf-internal-testing/tiny-random-distilbert')
# model = tfms.AutoModelForMaskedLM.from_config(config)
model = tfms.AutoModelForMaskedLM.from_pretrained('hf-internal-testing/tiny-random-distilbert')

Some weights of the model checkpoint at hf-internal-testing/tiny-random-distilbert were not used when initializing DistilBertForMaskedLM: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
eli5 = load_dataset("eli5", split="train_asks[:5000]")

Reusing dataset eli5 (/home/tornikeo/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [8]:
eli5 = eli5.train_test_split(test_size=0.2)

In [9]:
eli5["train"][0]

{'q_id': 'b5pfu3',
 'title': "What's the typical human number transcription error rate?",
 'selftext': "I've recently automated a process of finding prices in pdf invoices and typing then into a spreadsheet. \n\nIt is a lot faster than humans, but also is more accurate. It's there an estimate for error rate of humans when doing this type of reppetive work over thousands of iterations?",
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['ejhqxvp'],
  'text': ["Although there is probably some research on this done (mostly by corporations I'll guess), you seem to be in a perfect spot to experiment and determine this answer yourself. You could re-run a set of pdf files through your software and see how the humans fared in the past."],
  'score': [2]},
 'title_urls': {'url': []},
 'selftext_urls': {'url': []},
 'answers_urls': {'url': []}}

In [10]:
tokenizer = tfms.AutoTokenizer.from_pretrained('hf-internal-testing/tiny-random-distilbert')

In [11]:
eli5 = eli5.flatten()
eli5['train'][0]

{'q_id': 'b5pfu3',
 'title': "What's the typical human number transcription error rate?",
 'selftext': "I've recently automated a process of finding prices in pdf invoices and typing then into a spreadsheet. \n\nIt is a lot faster than humans, but also is more accurate. It's there an estimate for error rate of humans when doing this type of reppetive work over thousands of iterations?",
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['ejhqxvp'],
 'answers.text': ["Although there is probably some research on this done (mostly by corporations I'll guess), you seem to be in a perfect spot to experiment and determine this answer yourself. You could re-run a set of pdf files through your software and see how the humans fared in the past."],
 'answers.score': [2],
 'title_urls.url': [],
 'selftext_urls.url': [],
 'answers_urls.url': []}

In [24]:
eli5["train"].column_names

['q_id',
 'title',
 'selftext',
 'document',
 'subreddit',
 'answers.a_id',
 'answers.text',
 'answers.score',
 'title_urls.url',
 'selftext_urls.url',
 'answers_urls.url']

In [25]:
eli5.column_names

{'train': ['q_id',
  'title',
  'selftext',
  'document',
  'subreddit',
  'answers.a_id',
  'answers.text',
  'answers.score',
  'title_urls.url',
  'selftext_urls.url',
  'answers_urls.url'],
 'test': ['q_id',
  'title',
  'selftext',
  'document',
  'subreddit',
  'answers.a_id',
  'answers.text',
  'answers.score',
  'title_urls.url',
  'selftext_urls.url',
  'answers_urls.url']}

In [32]:
def preprocess_function(examples):
  x = tokenizer([' '.join(ex) for ex
                   in examples['answers.text']],
                   truncation=True,
                   padding=True,
#                    return_tensors='np'
                  )
  print({k:(len(v),len(v[0]), 'to', len(v[-1])) for k,v in x.items()})
  return x

tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    remove_columns=eli5["train"].column_names,
#     num_proc=4,
)

  0%|          | 0/4 [00:00<?, ?ba/s]

{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}
{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}
{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}
{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}


  0%|          | 0/1 [00:00<?, ?ba/s]

{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}


In [33]:
tokenized_eli5

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [34]:
block_size = 128

z = None
def group_texts(examples):
  global z
  #     print({k:(len(v),len(v[0])) for k,v in examples.items()})
  #     print({k:v[0][-1] for k,v in examples.items()})
  try:
#     concatenated_examples = {k: sum(examples[k], []) 
#                              for k in examples.keys()}
    print(list(examples.keys())[:4])
    concatenated_examples = {k: sum(examples[k], []) 
                             for k in examples.keys()}
    #     print({k:(len(v)) for k,v in concatenated_examples.items()})
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    print({k:(len(v)) for k,v in concatenated_examples.items()})

    result = {
        k: [t[i : i + block_size] 
            for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

  except TypeError as e:
    z = examples
#     raise e
    print(e)
    return None
lm_dataset = tokenized_eli5.map(group_texts, 
                                batched=True,) 
#                                 num_proc=4)

  0%|          | 0/4 [00:00<?, ?ba/s]

['input_ids', 'attention_mask']
{'input_ids': 512000, 'attention_mask': 512000}
['input_ids', 'attention_mask']
{'input_ids': 512000, 'attention_mask': 512000}
['input_ids', 'attention_mask']
{'input_ids': 512000, 'attention_mask': 512000}
['input_ids', 'attention_mask']
{'input_ids': 512000, 'attention_mask': 512000}


  0%|          | 0/1 [00:00<?, ?ba/s]

['input_ids', 'attention_mask']
{'input_ids': 512000, 'attention_mask': 512000}


In [35]:
lm_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 16000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4000
    })
})

In [ ]:
# x = tfms.AutoTokenizer.from_pretrained('distilbert-base-cased')

In [36]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [37]:
# tokenizer.pad_token = '[PAD]'
data_collator = tfms.DataCollatorForLanguageModeling(
  tokenizer=tokenizer,
  mlm_probability=.15,
)

In [38]:
training_args = tfms.TrainingArguments(
  output_dir='./results',
  evaluation_strategy="epoch",
  learning_rate=2e-5,
  num_train_epochs=3,
  weight_decay=0.01,
)

trainer = tfms.Trainer(
  model=model,
  args=training_args,
  train_dataset=lm_dataset['train'],
  eval_dataset=lm_dataset['test'],
  data_collator=data_collator,
)

In [39]:
trainer.train()

/home/tornikeo/.miniconda3/envs/torch/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 16000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6000


Epoch,Training Loss,Validation Loss


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


KeyboardInterrupt: 